# Point Cloud Demonstrations

## Registration

Registers two point clouds with slight offset.


In [1]:
import open3d as o3d
import numpy as np
import copy
import random

import utils

In [ ]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])

# load point clouds
base = o3d.io.read_point_cloud("./Point Clouds/Cleaner/1.ply")
source = o3d.io.read_point_cloud("./Point Clouds/Cleaner/8.ply")

# get 4x4 transformation matrix
trans = utils.register_clouds(base, source)

# show original clouds
o3d.visualization.draw_geometries([base, source])

# show aligned clouds
draw_registration_result_original_color(source, base, trans)


## Normals of point cloud
Shows the histogram of normals for the cloud. See utils module - running in notebook is much slower

## Outlier removal
Removes outliers and scatter artifacts from the pointcloud

In [ ]:
cl, ind=utils.clean_cloud("./Point Clouds/Cleaner/1.ply", view=True)

## Cleaned and combined 8 clouds

In [8]:
base = o3d.io.read_point_cloud("./Point Clouds/combined_cloud_clean.ply")
o3d.visualization.draw_geometries([base])

# Region Growing Segmentation
Demonstrates region growing algorithm for downsampled cloud

In [3]:
cl = o3d.io.read_point_cloud("./Point Clouds/combined_cloud_clean.ply")
pcd = cl.voxel_down_sample(0.01)

planes_list = utils.region_grow(pcd)

for plane in planes_list:
    colour = [random.uniform(0,1),random.uniform(0,1),random.uniform(0,1)]
    for i in plane:
        pcd.colors[i] = colour

o3d.visualization.draw_geometries([pcd])

# extract walls which is the largest object in the cloud

walls = planes_list[np.argmax(np.array([len(i) for i in planes_list]))]
walls = pcd.select_down_sample(walls)

## Isolating each wall plane

In [16]:
planes_list = utils.region_grow(walls, 0.8, True)

colours = [[1,0,0],[0,1,0],[0,0,1]]

for plane, colour in zip(planes_list, colours):
    for i in plane:
        walls.colors[i] = colour

o3d.visualization.draw_geometries([walls])

# get normals for each plane
plane_normals = []
for plane in planes_list:
    # make a copy otherwise points a deleted from selection
    wall_cp = copy.deepcopy(walls)
    plane_points = np.asarray(wall_cp.select_down_sample(plane).points)
    
    pseudoinverse = np.linalg.pinv(plane_points.T)
    plane_normals.append(pseudoinverse.T.dot(np.ones(plane_points.shape[0])))
    
plane_normals = np.array(plane_normals)
print(plane_normals)


[[ 0.95340822  0.27349228 -0.91945288]
 [ 0.17261405 -4.53552085 -1.09303153]
 [-0.94526643  0.23072868 -1.01467186]
 [-0.50359671 -2.44032973 -1.2665506 ]
 [-0.94230381  0.22407071 -1.01696776]
 [ 0.95060673  0.27444524 -0.92032463]
 [ 0.95127741  0.26911789 -0.92132911]
 [-0.94038304  0.22471468 -1.01726053]
 [ 0.1764165  -4.53207684 -1.09414902]
 [ 0.1733897  -4.53277351 -1.09308729]
 [ 0.26248531 -1.81458917 -1.30397151]
 [ 0.4750759  -1.88806846 -1.20529834]
 [-0.94205301  0.23587632 -1.0144949 ]
 [-0.73882852 -3.13813337 -1.27109101]]


## SVD of Walls
Principle vectors pick out the normals of the wall

In [15]:
plane = planes_list[1]
wall_cp = copy.deepcopy(walls)
plane = wall_cp.select_down_sample(plane)
points = np.asarray(plane.points)
points -= np.mean(points, axis=0)
u, s, v = np.linalg.svd(points)

o3d.visualization.draw_geometries([plane, utils.create_vector_graph(v)])

## Match to plane in full size cloud

In [4]:
# now match to the original full size point cloud

cl.paint_uniform_color([0.5,0.5,0.5])
for norm, colour in zip(plane_normals, colours):
    dot_match = np.abs(1-np.asarray(cl.points).dot(norm))
    for i in range(len(dot_match)):
        if dot_match[i] <0.04:
            cl.colors[i] = colour

# add normals for clearer viewing
if not cl.has_normals():
    cl.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.04 * 2, max_nn=30))
o3d.visualization.draw_geometries([cl])

## Merging model clouds


In [4]:
cl1 = o3d.io.read_point_cloud("./Point Clouds/Field Clouds/Commander/1.3.ply")
cl1, plane1 = utils.isolate_model(cl1)
cl1 += plane1

for i in range(4):
    print(i)
    cl2 = o3d.io.read_point_cloud("./Point Clouds/Field Clouds/Commander/1.{}.ply".format(i+2))
    cl2, plane2 = utils.isolate_model(cl2)
    temp_cl = cl2 + plane2

    tran = utils.register_clouds(cl1, temp_cl, voxel_radius=[0.01, 0.005, 0.0001], max_iter=[100,100,10000])

    cl2 = cl2.transform(tran)
    cl1 += cl2
    cl1.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.04 * 2, max_nn=30))
    
o3d.visualization.draw_geometries([cl1])

0
1
2
3
